# Manutenzione preventiva - DoS Attacks

In [1]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
from datetime import timedelta
from scipy.stats import entropy


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### costanti
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS15m-0,5h_1.json'

levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']

In [3]:
### estrazione dati
# packets clean capture
packets_cc = fn.load_packets_from_file(clean_capture)

# packets attack captures (es. packets ping flooding attack 1m per cattura di 0.5 = pkt_pf_1m_05h)
pkt_pf_15m_05h = fn.load_packets_from_file(pf_capture_1)

In [4]:
### features
# clean features
features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)

# ping flooding attack features
features_pf_1m_05h = fn.extract_features_from_packets(pkt_pf_15m_05h, levels_of_interest)

In [5]:
df_cc = pd.DataFrame(features_cc)
df = pd.DataFrame(features_pf_1m_05h)

In [6]:
features_cc.clear()
features_pf_1m_05h.clear()

In [7]:
fn.calcolo_features_binarie(df_cc)
fn.calcolo_features_binarie(df)

In [8]:
df_cc['frame.frame.time_utc'] = pd.to_datetime(df_cc['frame.frame.time_utc'])
df['frame.frame.time_utc'] = pd.to_datetime(df['frame.frame.time_utc'])

In [9]:
df_cc.sort_values(by='frame.frame.time_utc')
df.sort_values(by='frame.frame.time_utc')

,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len,...,llc,mbtcp,icmpv6,ip,icmp,modbus,tcp,stp,arp,eth
0,1,"May 21, 2018 13:12:37.348723000 ora legale Eur...",2018-05-21 11:12:37.348723+00:00,1526901157.348723000,0.000000000,0.000000000,0.000000000,0.000000000,1,60,...,0,0,0,0,0,0,0,0,1,1
1,1,"May 21, 2018 13:12:37.350527000 ora legale Eur...",2018-05-21 11:12:37.350527+00:00,1526901157.350527000,0.000000000,0.001804000,0.001804000,0.001804000,2,64,...,0,0,0,0,0,0,0,0,1,1
2,1,"May 21, 2018 13:12:37.350544000 ora legale Eur...",2018-05-21 11:12:37.350544+00:00,1526901157.350544000,0.000000000,0.000017000,0.000017000,0.001821000,3,60,...,0,0,0,1,0,0,1,0,0,1
3,1,"May 21, 2018 13:12:37.360801000 ora legale Eur...",2018-05-21 11:12:37.360801+00:00,1526901157.360801000,0.000000000,0.010257000,0.010257000,0.012078000,4,60,...,0,0,0,1,0,0,1,0,0,1
4,1,"May 21, 2018 13:12:37.360805000 ora legale Eur...",2018-05-21 11:12:37.360805+00:00,1526901157.360805000,0.000000000,0.000004000,0.000004000,0.012082000,5,60,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197773,1,"May 21, 2018 13:42:36.345657000 ora legale Eur...",2018-05-21 11:42:36.345657+00:00,1526902956.345657000,0.000000000,0.001220000,0.001220000,1798.996934000,197774,60,...,0,0,0,1,0,0,1,0,0,1
197774,1,"May 21, 2018 13:42:36.345673000 ora legale Eur...",2018-05-21 11:42:36.345673+00:00,1526902956.345673000,0.000000000,0.000016000,0.000016000,1798.996950000,197775,60,...,0,0,0,1,0,0,1,0,0,1
197775,1,"May 21, 2018 13:42:36.345854000 ora legale Eur...",2018-05-21 11:42:36.345854+00:00,1526902956.345854000,0.000000000,0.000181000,0.000181000,1798.997131000,197776,60,...,0,0,0,1,0,0,1,0,0,1
197776,1,"May 21, 2018 13:42:36.354401000 ora legale Eur...",2018-05-21 11:42:36.354401+00:00,1526902956.354401000,0.000000000,0.008547000,0.008547000,1799.005678000,197777,60,...,0,0,0,1,0,0,1,0,0,1


In [10]:
start_time = df['frame.frame.time_utc'].iloc[0]
end_time=start_time+timedelta(seconds=298.8)
end_time2 = end_time + timedelta(seconds=900)
end_time

Timestamp('2018-05-21 11:17:36.148723+0000', tz='UTC')

In [11]:
df_cc1 = df[(df['frame.frame.time_utc'] >= start_time) & (df['frame.frame.time_utc'] < end_time)].copy()
df_attack = df[(df['frame.frame.time_utc'] >= end_time) & (df['frame.frame.time_utc'] <= end_time2)].copy()
df_cc2 = df[(df['frame.frame.time_utc'] > end_time2)].copy()

In [12]:
df_cc['frame.frame.time_utc'] = pd.to_datetime(df_cc['frame.frame.time_utc'], unit='s')
df_cc_aggregation = pd.DataFrame()
start_date = pd.Timestamp(df_cc['frame.frame.time_utc'].min())
end_date = pd.Timestamp(df_cc['frame.frame.time_utc'].max() + pd.Timedelta(seconds=5))
dates_enumeration = pd.date_range(start=start_date, end=end_date, freq='5S')
df_cc_aggregation['timestamp'] = dates_enumeration

In [13]:
# Numero di eth, ip, arp, tcp, udp, tcp, mbtcp per raggruppamento nella cattura clean
df_cc_aggregation['eth_count']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['eth'].apply(lambda x: (x==1).sum()).reset_index(name='count')['count']
df_cc_aggregation['ip_count']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip'].apply(lambda x: (x==1).sum()).reset_index(name='count')['count']
df_cc_aggregation['arp_count']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['arp'].apply(lambda x: (x==1).sum()).reset_index(name='count')['count']
df_cc_aggregation['udp_count']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['udp'].apply(lambda x: (x==1).sum()).reset_index(name='count')['count']
df_cc_aggregation['tcp_count']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp'].apply(lambda x: (x==1).sum()).reset_index(name='count')['count']
df_cc_aggregation['mbtcp_count']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['mbtcp'].apply(lambda x: (x==1).sum()).reset_index(name='count')['count']
if 'icmp' in df_cc.columns:
	df_cc_aggregation['icmp_count']=df.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['icmp'].apply(lambda x: (x==1).sum()).reset_index(name='count')['count']
else:
	df_cc_aggregation['icmp_count']=0
# Numero di pacchetti per cattura
df_cc_aggregation['pkt_count']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['frame.frame.time_utc'].count().reset_index(name='pkt_count')['pkt_count']
df_cc_aggregation['per_eth_count'] = df_cc_aggregation['eth_count']/df_cc_aggregation['pkt_count']
df_cc_aggregation['per_ip_count'] = df_cc_aggregation['ip_count']/df_cc_aggregation['pkt_count']
df_cc_aggregation['per_icmp_count'] = df_cc_aggregation['icmp_count']/df_cc_aggregation['pkt_count']
df_cc_aggregation['per_arp_count'] = df_cc_aggregation['arp_count']/df_cc_aggregation['pkt_count']
df_cc_aggregation['per_udp_count'] = df_cc_aggregation['udp_count']/df_cc_aggregation['pkt_count']
df_cc_aggregation['per_tcp_count'] = df_cc_aggregation['tcp_count']/df_cc_aggregation['pkt_count']
df_cc_aggregation['per_mbtcp_count'] = df_cc_aggregation['mbtcp_count']/df_cc_aggregation['pkt_count']

In [14]:
# Conversione dei flags del tcp
# SYN
df_cc['tcp.tcp.flags_tree.tcp.flags.syn'].fillna('0', inplace=True)
df_cc['tcp.tcp.flags_tree.tcp.flags.syn'] = df_cc['tcp.tcp.flags_tree.tcp.flags.syn'].astype(int)
# ACK
df_cc['tcp.tcp.flags_tree.tcp.flags.ack'].fillna('0', inplace=True)
df_cc['tcp.tcp.flags_tree.tcp.flags.ack'] = df_cc['tcp.tcp.flags_tree.tcp.flags.ack'].astype(int)

In [15]:
# Aggiunta colonne tcp flags aggregate per raggruppamento
df_cc_aggregation['tcp_syn_count']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp.tcp.flags_tree.tcp.flags.syn'].apply(lambda x: (x==1).sum()).reset_index(name='count')['count']
df_cc_aggregation['tcp_ack_count']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp.tcp.flags_tree.tcp.flags.ack'].apply(lambda x: (x==1).sum()).reset_index(name='count')['count']
df_cc_aggregation['tcp_synack_fraction'] = df_cc_aggregation['tcp_syn_count']/df_cc_aggregation['tcp_ack_count']

In [16]:
# Calcolo dell'inter arrival time
df_cc['inter.packet_arrival_time'] = df_cc['frame.frame.time_utc'].diff()

In [17]:
# Calcolo delle features aggregate su inter-arrival-time
df_cc_aggregation['ipat_std']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['inter.packet_arrival_time'].apply(lambda x: x.std()).reset_index(name='count')['count']
df_cc_aggregation['ipat_mode']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['inter.packet_arrival_time'].apply(lambda x: x.mode()).reset_index(name='count')['count']
df_cc_aggregation['ipat_max']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['inter.packet_arrival_time'].apply(lambda x: x.max()).reset_index(name='count')['count']
df_cc_aggregation['ipat_min']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['inter.packet_arrival_time'].apply(lambda x: x.min()).reset_index(name='count')['count']
df_cc_aggregation['ipat_entropy']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['inter.packet_arrival_time'].apply(lambda x: entropy(x.value_counts()/len(x))).reset_index(name='count')['count']

In [18]:
# Sostituzione dei NaN con un valore relativo a indirizzo di rete che non viene mai utilizzato nel dataset e conversione dell'ip src in intero
df_cc['ip.ip.src'].fillna('169.254.0.0', inplace=True)
df_cc['ip.ip.src_asint'] = df_cc['ip.ip.src'].apply(lambda x: int(ip.IPv4Address(x.strip())))

In [19]:
# Calcolo delle features aggregate su ip.ip.src
df_cc_aggregation['ip_src_std']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip.ip.src_asint'].apply(lambda x: x.std()).reset_index(name='count')['count']
df_cc_aggregation['ip_src_mode']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip.ip.src_asint'].apply(lambda x: x.mode()).reset_index(name='count')['count']
df_cc_aggregation['ip_src_entropy']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip.ip.src_asint'].apply(lambda x: entropy(x.value_counts()/len(x))).reset_index(name='count')['count']

In [20]:
# Sostituzione dei NaN con un valore relativo a indirizzo di rete che non viene mai utilizzato nel dataset e conversione dell'ip dst in intero
df_cc['ip.ip.dst'].fillna('169.254.0.0', inplace=True)
df_cc['ip.ip.dst_asint'] = df_cc['ip.ip.dst'].apply(lambda x: int(ip.IPv4Address(x.strip())))

In [21]:
# Calcolo delle features aggregate su ip.ip.dst
df_cc_aggregation['ip_dst_std']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip.ip.dst_asint'].apply(lambda x: x.std()).reset_index(name='count')['count']
df_cc_aggregation['ip_dst_mode']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip.ip.dst_asint'].apply(lambda x: x.mode()).reset_index(name='count')['count']
df_cc_aggregation['ip_dst_entropy']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip.ip.dst_asint'].apply(lambda x: entropy(x.value_counts()/len(x))).reset_index(name='count')['count']

In [22]:
# Sostituzione dei NaN con un valore relativo a una porta mai utilizzata e conversione della colonna tcp.tcp.dstport in intero
df_cc['tcp.tcp.dstport'].fillna('0', inplace=True)
df_cc['tcp.tcp.dstport_asint'] = df_cc['tcp.tcp.dstport'].apply(lambda x: int(x))

In [78]:
# Calcolo delle features aggregate su tcp.tcp.dst
df_cc_aggregation['tcp_dstport_std']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp.tcp.dstport_asint'].apply(lambda x: x.std()).reset_index(name='count')['count']
df_cc_aggregation['tcp_dstport_mode']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp.tcp.dstport_asint'].apply(lambda x: x.mode()).reset_index(name='count')['count']
df_cc_aggregation['tcp_dstport_entropy']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp.tcp.dstport_asint'].apply(lambda x: entropy(x.value_counts()/len(x))).reset_index(name='count')['count']
df_cc_aggregation['tcp_dstport_mbtcp']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp.tcp.dstport_asint'].apply(lambda x: (x==502).sum()).reset_index(name='count')['count']

In [24]:
# Sostituzione dei NaN con un valore relativo a una porta mai utilizzata e conversione della colonna tcp.tcp.srcport in intero
df_cc['tcp.tcp.srcport'].fillna('0', inplace=True)
df_cc['tcp.tcp.srcport_asint'] = df_cc['tcp.tcp.srcport'].apply(lambda x: int(x))

In [25]:
# Calcolo delle features aggregate su tcp.tcp.src
df_cc_aggregation['tcp_srcport_std']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp.tcp.srcport_asint'].apply(lambda x: x.std()).reset_index(name='count')['count']
df_cc_aggregation['tcp_srcport_mode']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp.tcp.srcport_asint'].apply(lambda x: x.mode()).reset_index(name='count')['count']
df_cc_aggregation['tcp_srcport_entropy']=df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['tcp.tcp.srcport_asint'].apply(lambda x: entropy(x.value_counts()/len(x))).reset_index(name='count')['count']

In [47]:
# CALCOLA NUMERO DI PACCHETTI CON STESSO IP DI DESTINAZIONE, MA GLI IP DI DESTINAZIONE POSSONO ESSERE DIVERSI, QUINDI IL DATAFRAME AVRA' PIù RIGHE
df_cc_1 = df_cc.groupby([pd.Grouper(key='frame.frame.time_utc', freq='5s'), 'ip.ip.dst_asint']).size().reset_index(name='num_unique_ipdst')
df_cc_aggregation['maxnum_unique_ipdst'] = df_cc_1.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['num_unique_ipdst'].max().reset_index(name='maxnum_unique_ipdst')['maxnum_unique_ipdst']
df_cc_aggregation['minnum_unique_ipdst'] = df_cc_1.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['num_unique_ipdst'].min().reset_index(name='minnum_unique_ipdst')['minnum_unique_ipdst']
df_cc_aggregation['modenum_unique_ipdst'] = df_cc_1.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['num_unique_ipdst'].apply(lambda x: x.mode()).reset_index(name='modenum_unique_ipdst')['modenum_unique_ipdst']
df_cc_aggregation['stdnum_unique_ipdst'] = df_cc_1.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['num_unique_ipdst'].apply(lambda x: x.std()).reset_index(name='stdnum_unique_ipdst')['stdnum_unique_ipdst']
df_cc_aggregation['entropynum_unique_ipdst'] = df_cc_1.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['num_unique_ipdst'].apply(lambda x: entropy(x.value_counts()/len(x))).reset_index(name='entropynum_unique_ipdst')['entropynum_unique_ipdst']

In [62]:
# Numero di ip sorgenti/destinazione univoci nel raggruppamento
df_cc_aggregation['num_unique_ipsrc'] = df_cc.groupby([pd.Grouper(key='frame.frame.time_utc', freq='5s')])['ip.ip.src'].nunique().reset_index(name='num_unique_ipsrc')['num_unique_ipsrc']
df_cc_aggregation['num_unique_ipdst'] = df_cc.groupby([pd.Grouper(key='frame.frame.time_utc', freq='5s')])['ip.ip.dst'].nunique().reset_index(name='num_unique_ipdst')['num_unique_ipdst']

In [60]:
# Sostituzione valori nulli e conversione del campo ip.len in intero
df_cc['ip.ip.len'].fillna('0', inplace=True)
df_cc['ip.ip.len'] = df_cc['ip.ip.len'].astype(int)

In [63]:
# Calcolo aggregazioni su campo ip.len
## La moda è stata eliminata poichè ogni riga assumeva valore 40
df_cc_aggregation['std_iplen'] = df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip.ip.len'].apply(lambda x: x.std()).reset_index(name='std_iplen')['std_iplen']
df_cc_aggregation['entropy_iplen'] = df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip.ip.len'].apply(lambda x: entropy(x.value_counts()/len(x))).reset_index(name='entropy_iplen')['entropy_iplen']

In [66]:
# Calcolo numero di bytes per unita di tempo
df_cc_aggregation['bytes_per_timeunit'] = df_cc.groupby(pd.Grouper(key='frame.frame.time_utc', freq='5s'))['ip.ip.len'].apply(lambda x: x.sum()/5).reset_index(name='bytes_per_timeunit')['bytes_per_timeunit']

In [70]:
# Calcolo numero di pacchetti per unita di tempo
df_cc_aggregation['pkt_per_timeunit'] = df_cc_aggregation['pkt_count']/5

In [86]:
df_cc['modbus.modbus.func_code'].fillna('67', inplace=True)
df_cc['modbus.modbus.func_code'] = df_cc['modbus.modbus.func_code'].astype(int)